In [ ]:
import random

import matplotlib
matplotlib.use('Qt5Agg')

import matplotlib.pyplot as pt
from matplotlib.animation import FuncAnimation

class Pacjent:
    def __init__(self, status, x, y):
        if status != 'chory' and status != 'zdrowy' and status != 'nosiciel' and status != 'martwy' and status != 'odporny':
            raise ValueError("Zły status pacjenta")
        self.status = status 
        self.x = x
        self.y = y
        self.t = 0
        self.predkosc_x = random.uniform(-0.5,0.5)
        self.predkosc_y = random.uniform(-0.5,0.5)
    def ruch(self):
        if self.status == 'zdrowy' or self.status == 'nosiciel' or self.status == 'odporny':
            self.predkosc_x = self.predkosc_x + random.uniform(-0.25,0.25)
            self.predkosc_y = self.predkosc_y + random.uniform(-0.25,0.25)
            if self.predkosc_x > 1:
                self.predkosc_x = 1
            if self.predkosc_y > 1:
                self.predkosc_y = 1
        if self.status == 'chory':
            self.predkosc_x = self.predkosc_x + random.uniform(-0.1,0.1)
            self.predkosc_y = self.predkosc_y + random.uniform(-0.1,0.1)
            if self.predkosc_x > 0.2:
                self.predkosc_x = 0.2
            if self.predkosc_y > 0.2:
                self.predkosc_y = 0.2
        if self.status == 'martwy':
            self.predkosc_x = 0
            self.predkosc_y = 0
        self.x = self.x + self.predkosc_x
        self.y = self.y + self.predkosc_y
    def zapis(self, nazwa):
        plik = open(nazwa, "w+")
        plik.write(self.nazwa,self.x,self.y)
        plik.close()
    def dopis(self, nazwa):
        plik = open(nazwa, "a+")
        plik.write(self.nazwa,self.x,self.y)
        plik.close()
    def odczyt(self, nazwa):
        plik = open(nazwa, "r")
        tmp = plik.read()
        status = plik[0]
        x = plik[1]
        y = plik[2]
        plik.close()
        
            
class Populacja:
    def __init__(self, n = 100, szer = 100, wys = 100, dystans = 3, czas = 20, smier = 0.5, ilosc = 0.05):
        self.n = n
        self.szer = szer
        self.wys = wys
        self.pacjenci = []
        self.dystans = dystans
        self.czas = czas
        self.smier = smier
        self.t = 0
        self.historia_zarazen = []
        self.ilosc = ilosc
        for i in range(0,self.n):
            choroba = random.uniform(0,1)
            if choroba > ilosc:
                choroba = 'zdrowy'
            else:
                choroba = 'chory'
            x = random.uniform(0,self.szer)
            y = random.uniform(0,self.wys)
            self.pacjenci.append(Pacjent(choroba,x,y))
    def odleglosc(self, i, j):
        return ((self.pacjenci[i].x-self.pacjenci[j].x)**2 + (self.pacjenci[i].y-self.pacjenci[j].y)**2)**(1/2)
    def ruch(self):
        self.t = self.t + 1 
        for i in range (0,self.n):
            self.pacjenci[i].ruch()
            if self.pacjenci[i].x > self.szer:
                self.pacjenci[i].x = self.pacjenci[i].x - self.szer
            if self.pacjenci[i].x < 0:
                self.pacjenci[i].x = self.pacjenci[i].x + self.szer
            if self.pacjenci[i].y > self.wys:
                self.pacjenci[i].y = self.pacjenci[i].y - self.wys
            if self.pacjenci[i].y < 0:
                self.pacjenci[i].y = self.pacjenci[i].y + self.wys
        for i in range (0,self.n):
            for j in range (0,self.n):
                if i != j and self.odleglosc(i, j)<self.dystans and (self.pacjenci[j].status == 'chory' or self.pacjenci[j].status == 'nosiciel') and self.pacjenci[i].status == 'zdrowy':
                    self.historia(self.pacjenci[i].x,self.pacjenci[i].y)
                    los = random.uniform(0,1)
                    if los > 0.5:
                        self.pacjenci[i].status = 'chory'
                    else:
                        self.pacjenci[i].status = 'nosiciel'
        for i in range (0,self.n):
            if self.pacjenci[i].status == 'chory' or self.pacjenci[i].status == 'nosiciel':
                self.pacjenci[i].t = self.pacjenci[i].t + 1
            if self.pacjenci[i].t > self.czas:
                if self.pacjenci[i].status == 'chory':
                    los = random.uniform(0,1)
                    if los > self.smier:
                        self.pacjenci[i].status = 'odporny'
                    else:
                        self.pacjenci[i].status = 'martwy'
                elif self.pacjenci[i].status != 'martwy':
                    self.pacjenci[i].status = 'odporny'
    def historia(self, x, y):
        self.historia_zarazen.append((self.t,x,y))
    def zapis(self, nazwa):
        plik = open(nazwa, "w+")
        for i in range (0,self.n):
            plik.write(self.pacjenci[i].status, self.pacjenci[i].x, self.pacjenci[i].y)
        plik.close()  
    def wypisz(self):
        for i in range(0,self.n):
            print("Polozenie pacjenta nr",i,"to",self.pacjenci[i].x,",",self.pacjenci[i].y,", a jego status to",self.pacjenci[i].status)

zdrowi = []
chorzy = []
nosiciele = []
odporni = []
martwi = []
            
def rysuj(populacja):
    zdrowi_x = []
    zdrowi_y = []
    chorzy_x = []
    chorzy_y = []
    nosiciele_x = []
    nosiciele_y = []
    odporni_x = []
    odporni_y = []
    martwi_x = []
    martwi_y = []
    for i in range(0,populacja.n):
        if populacja.pacjenci[i].status == 'zdrowy':
            zdrowi_x.append(populacja.pacjenci[i].x)
            zdrowi_y.append(populacja.pacjenci[i].y)
        if populacja.pacjenci[i].status == 'chory':
            chorzy_x.append(populacja.pacjenci[i].x)
            chorzy_y.append(populacja.pacjenci[i].y)
        if populacja.pacjenci[i].status == 'nosiciel':
            nosiciele_x.append(populacja.pacjenci[i].x)
            nosiciele_y.append(populacja.pacjenci[i].y)
        if populacja.pacjenci[i].status == 'odporny':
            odporni_x.append(populacja.pacjenci[i].x)
            odporni_y.append(populacja.pacjenci[i].y)
        if populacja.pacjenci[i].status == 'martwy':
            martwi_x.append(populacja.pacjenci[i].x)
            martwi_y.append(populacja.pacjenci[i].y)
#    a, = pt.plot(zdrowi_x, zdrowi_y, color='green', marker = '.', linestyle ='')
#    b, = pt.plot(chorzy_x, chorzy_y, color='red', marker = '.', linestyle ='')
#    c, = pt.plot(nosiciele_x, nosiciele_y, color='yellow', marker = '.', linestyle ='')
    zdrowi.append(len(zdrowi_x))
    chorzy.append(len(chorzy_x))
    nosiciele.append(len(nosiciele_x))
    odporni.append(len(odporni_x))
    martwi.append(len(martwi_x))
    return pt.plot(zdrowi_x, zdrowi_y, '. g',chorzy_x, chorzy_y , '. r', nosiciele_x, nosiciele_y, '. y', odporni_x, odporni_y, '. b', martwi_x, martwi_y, '. k')

a = int(input("Podaj populacje: "))
b = float(input("Podaj szerokosc: "))
c = float(input("Podaj wysokosc: "))
d = float(input("Podaj zasieg zarazliwosci: "))
e = float(input("Podaj czas trwania choroby: "))
f = float(input("Podaj smiertelnosc: "))
g = float(input("Podaj poczatkowy stosunek chorych"))

populacja = Populacja(a, b, c, d, e, f, g)
#populacja.wypisz()
#rysuj(populacja)


fig, ax = pt.subplots()
ln = rysuj(populacja)

def init():
    ax.set_xlim(0, populacja.szer)
    ax.set_ylim(0, populacja.wys)
    return ln

def update(frame):
    populacja.ruch()
    ln = rysuj(populacja)
    return ln

ani = FuncAnimation(fig, update, frames=None, init_func=init, blit=True, interval = 10)
pt.show(block=True)

#for i in range (0, len(zdrowi)):
#    print (zdrowi[i], nosiciele[i], chorzy[i], odporni[i], martwi[i])

#for i in range(0,len(populacja.historia_zarazen)):
#    print(populacja.historia_zarazen[i])